## 라이브러리 설치 및 드라이브 마운트

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
pip install html2text

## 상세 크롤링 코드

In [2]:
import requests
from bs4 import BeautifulSoup
import json
import re
import html2text

def crawl_post_content(url):
    """단일 게시글 URL에서 본문 텍스트, 이미지, 링크를 추출하여 반환"""

    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # 게시글 전체 영역 지정
        post_wrapper = soup.find('div', class_='bg-white p-4 mb-5')
        if not post_wrapper:
            return None

        # --- 메타데이터 추출 (옵션) ---
        # --- (생략: 메타데이터 추출 부분 - 이 함수에서는 반환하지 않고 목록 크롤러에서 사용) ---


        # --- 본문 영역 지정 (HR 태그 이후) ---
        # hr 태그를 찾고, 그 다음부터 본문이 시작되는 div를 찾습니다.
        hr_tag = post_wrapper.find('hr', class_='mb-4 mt-4')

        # hr_tag 바로 다음 형제(sibling) 요소를 본문 컨테이너로 간주
        content_container = hr_tag.find_next_sibling('div') if hr_tag else None

        if not content_container:
            print(f"경고: 본문 콘텐츠 영역을 찾을 수 없습니다. URL: {url}")
            return {'full_body_markdown': '', 'image_urls': [], 'download_links': []}

        # --- 본문 내용 추출 및 데이터 수집 ---
        image_urls = []
        download_links = []

       # --- A. 이미지 URL 추출 ---
        # Markdown 변환 전에 이미지 URL을 먼저 추출합니다.
        IMAGE_EXTENSIONS = r'\.(jpg|jpeg|png|gif|svg|webp)$'
        for img_tag in content_container.find_all('img'):
            img_src = img_tag.get('src')
            if img_src and re.search(IMAGE_EXTENSIONS, img_src, re.IGNORECASE) and img_src not in image_urls:
                # 상대 경로를 절대 경로로 변환
                image_urls.append(requests.compat.urljoin(url, img_src))

        # --- B. 다운로드 링크 추출 ---
        # 다운로드 리스트 구조를 명시적으로 찾습니다.
        for li in content_container.select('ul.download-list li a'):
            href = li.get('href')
            text_span = li.find('span')
            text = text_span.get_text(strip=True) if text_span else '다운로드 파일'

            if href:
                download_links.append({
                    'text': text,
                    'href': requests.compat.urljoin(url, href)
                })

        # --- C. HTML to Markdown 변환 (핵심) ---
        # 복잡한 테이블, 목록, 하이퍼링크 처리를 html2text에 맡깁니다.
        h = html2text.HTML2Text()
        h.ignore_links = False  # 링크를 [텍스트](URL) 마크다운으로 변환하도록 설정
        h.body_width = 0        # 줄 바꿈 강제하지 않음

        # content_container 내부의 HTML 조각을 Markdown으로 변환
        markdown_text = h.handle(str(content_container))

        # 최종 텍스트 정리: 과도한 줄바꿈만 정리
        final_markdown = re.sub(r'\n{3,}', '\n\n', markdown_text).strip()

        # 3. 최종 결과 반환
        return {
            'full_body_markdown': final_markdown, # 👈 필드명 변경 및 Markdown 텍스트
            'image_urls': image_urls,
            'download_links': download_links
        }

    except requests.exceptions.RequestException as e:
        print(f"상세 크롤링 오류 발생: {url}, 오류: {e}")
        return None
    except Exception as e:
        print(f"상세 크롤링 중 알 수 없는 오류: {url}, 오류: {e}")
        return None

# # 예시 사용법
# # data = crawl_post_content("https://scatch.ssu.ac.kr/%EA%B3%B5%EC%A7%80%EC%82%AC%ED%95%AD/?slug=2025%ed%95%99%eb%85%84%eb%8f%84-2%ed%95%99%ea%b8%b0-sic-4%ec%bf%bc%ed%84%b0-%ec%a0%95%ea%b8%b0-%ea%b5%90%ed%99%98%ed%95%99%ec%83%9d-%ec%84%b8%eb%af%b8%eb%82%98-saf")
# # print(json.dumps(data, ensure_ascii=False, indent=4))

### 날짜 형식 ISO로 변환

In [3]:
from datetime import datetime as dt
from datetime import date
import locale

def convert_korean_date_to_iso(korean_date_str):
    """
    'YYYY.MM.DD' 형식의 날짜 문자열을 ISO 8601 형식(YYYY-MM-DD)으로 변환합니다.
    """
    if not korean_date_str:
        return None

    # 형식: YYYY.MM.DD 또는 YYYY-MM-DD 등으로 들어올 수 있음.
    # 여기서는 '2025.11.04' 형식을 가정하고 파싱합니다.
    try:
        dt_object = dt.strptime(korean_date_str.strip().replace('/', '.').replace('-', '.'), '%Y.%m.%d')
        return dt_object.strftime('%Y-%m-%d')
    except ValueError:
        # 실패 시 None 반환
        return None

# --- 사용 예시 ---
# date_str_kor = '2025.11.04'
# iso_date = convert_korean_date_to_iso(date_str_kor)
# print(f"원본: {date_str_kor} -> ISO: {iso_date}") # 출력: 2025-11-04

## 메타데이터 및 상세데이터 추출 (상세데이터 추출 함수 이용)

### 크롤링 결과 파일 경로 및 이름 설정

크롤링 결과 예시

[레코드 #1]
  제목: 2026년도 (재)전기공사공제조합장학회 제21기 장학생 선발 안내

  URL: https://scatch.ssu.ac.kr/

  작성일: 2025-11-04

  부서: 장학팀

  조회수: 34회

  이미지 수: 1개

  첨부파일 수: 0개

  본문 미리보기: [![](https://scatch.ssu.ac.kr/wp-content/uploads/sites/5/2025/11/2026년도-재전기공사공제조합장학회-제21기-장학생-선발-공고.img)](https://scatch.ssu.ac.kr/wp-content/uploads/sites/5/2025/11/2026년도-재전기공사공제조합장학회-제21기-장학생-선발-공고...





In [6]:
FILE_NAME = '/content/drive/MyDrive/자연어/ssu_rag_data_2025_v2.jsonl'

In [7]:
from datetime import datetime as dt
from datetime import date

BASE_URL = "https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec%82%ac%ed%95%ad/"
TARGET_YEAR = 2025

def fetch_notices_all_pages(base_url, max_pages):
    """
    게시글 목록을 순회하며 2025년 이후의 항목만 필터링하여 메타데이터를 추출합니다.
    """
    all_notices_metadata = []

    # 날짜 필터링 기준 (2024년 12월 31일)
    cutoff_date = date(TARGET_YEAR - 1, 12, 31)

    for page_num in range(1, max_pages + 1):
        # URL 경로 설정: '/page/N/' 형태
        url = f"{base_url}page/{page_num}/" if page_num > 1 else base_url

        try:
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # 💡 정확한 선택자: 개별 게시글 항목을 나타내는 div
            notice_items = soup.select('div.row.no-gutters.align-items-center')

            if not notice_items:
                print(f"페이지 {page_num} ({url})에서 항목을 찾을 수 없어 종료합니다.")
                break

            should_break = False

            for item in notice_items:
                # 1. 게시일 추출 및 필터링
                date_element = item.select_one('.notice_col1 .h2')
                raw_date = date_element.get_text(strip=True) if date_element else None

                if raw_date:
                    try:
                        # 'YYYY.MM.DD' 형식을 datetime 객체로 파싱
                        post_date = dt.strptime(raw_date, '%Y.%m.%d').date()

                        # 2024년 이전 게시글이라면 순회 중단 플래그 설정
                        if post_date <= cutoff_date:
                            print(f"2024년 이전 게시글 발견 ({post_date}). 목록 순회 중단.")
                            should_break = True
                            break # item 순회 중단

                    except ValueError:
                        pass # 날짜 파싱 오류는 무시

                # 2. 2025년 이후 항목에 대해서만 메타데이터 추출
                try:
                   # 변동성 때문에 건너뜀.
                   # status_element = item.select_one('.notice_col2 .tag')
                   # status = status_element.text.strip() if status_element else 'N/A'

                    # 링크를 포함하는 요소 추출
                    link_element = item.select_one('.notice_col3 a')
                    link_href = link_element['href'].strip() if link_element and 'href' in link_element.attrs else 'N/A'

                    category_element = link_element.select_one('.label')
                    category = category_element.text.strip() if category_element else 'N/A'

                    title_span = link_element.select_one('span[class*="d-inline-b"].m-pt-5')
                    title = title_span.text.strip() if title_span else '제목 없음'

                    # 등록부서 추출 (notice_col4)
                    department_element = item.select_one('.notice_col4')
                    department = department_element.text.strip() if department_element else 'N/A'

                    # 조회수 추출 (notice_col5)
                    views_element = item.select_one('.notice_col5')
                    views = views_element.text.strip() if views_element else 'N/A'

                    all_notices_metadata.append({
                        '작성일': raw_date,
                        '카테고리': category,
                        '제목': title,
                        '링크': link_href,
                        '등록부서': department,
                        '조회수': views
                    })

                except Exception:
                    continue # 항목 파싱 오류 무시

            if should_break:
                break # 전체 페이지 순회 중단

            print(f"페이지 {page_num} 크롤링 완료. 다음 페이지로 진행합니다.")

        except requests.exceptions.RequestException as e:
            print(f"페이지 {page_num} ({url})를 가져오는 중 오류가 발생했습니다: {e}")
            break

    return all_notices_metadata


def crawl_and_save_rag_data(notices_metadata, filename):
    """
    수집된 목록 메타데이터를 순회하며 상세 크롤링을 수행하고 JSON Lines 파일로 저장합니다.
    """
    print(f"\n--- RAG 상세 크롤링 및 JSON Lines 파일 생성 시작 (총 {len(notices_metadata)}개) ---")

    with open(filename, 'w', encoding='utf-8') as f:
        for i, notice in enumerate(notices_metadata):
            link = notice['링크']

            # 1. 상세 크롤링 (본문 및 이미지 URL 획득)
            detailed_data = crawl_post_content(link)

            if not detailed_data:
                continue

            # 2. RAG 레코드 통합 및 포맷팅
            rag_record = {
                'source_url': link,
                'post_title': notice['제목'],
                'category': notice['카테고리'],
                # 'status': notice['진행상황'],
                'department': notice['등록부서'],
                'views': int(notice['조회수']),
                'posted_date': convert_korean_date_to_iso(notice['작성일']), # ISO 변환
                'full_body_markdown': detailed_data['full_body_markdown'],
                'image_urls': detailed_data['image_urls'],
                'download_links': detailed_data['download_links']
            }

            # 3. JSON Lines 파일에 저장
            f.write(json.dumps(rag_record, ensure_ascii=False) + '\n')

            if (i + 1) % 100 == 0:
                 print(f"✅ {i+1}번째 항목까지 JSON Lines 저장 완료.")

    print(f"\n🎉 RAG 데이터가 '{filename}' 파일에 성공적으로 저장되었습니다.")


if __name__ == '__main__':
    MAX_PAGES = 80

    print(f"--- 숭실대학교 공지사항 크롤링 시작 ({TARGET_YEAR}년 이후 필터링 및 페이지네이션) ---")

    metadata_results = fetch_notices_all_pages(BASE_URL, MAX_PAGES)

    if metadata_results:
        print(f"\n✅ 최종적으로 총 {len(metadata_results)}개의 ({TARGET_YEAR}년 이후) 공지사항 메타데이터를 수집했습니다.")

        # 2단계: 상세 크롤링 및 JSON Lines 저장
        crawl_and_save_rag_data(metadata_results, FILE_NAME)

    else:
        print("공지사항을 찾지 못했거나 크롤링에 실패했습니다. URL과 CSS Selector를 다시 확인해 주세요.")





--- 숭실대학교 공지사항 크롤링 시작 (2025년 이후 필터링 및 페이지네이션) ---
페이지 1 크롤링 완료. 다음 페이지로 진행합니다.
페이지 2 크롤링 완료. 다음 페이지로 진행합니다.
페이지 3 크롤링 완료. 다음 페이지로 진행합니다.
페이지 4 크롤링 완료. 다음 페이지로 진행합니다.
페이지 5 크롤링 완료. 다음 페이지로 진행합니다.
페이지 6 크롤링 완료. 다음 페이지로 진행합니다.
페이지 7 크롤링 완료. 다음 페이지로 진행합니다.
페이지 8 크롤링 완료. 다음 페이지로 진행합니다.
페이지 9 크롤링 완료. 다음 페이지로 진행합니다.
페이지 10 크롤링 완료. 다음 페이지로 진행합니다.
페이지 11 크롤링 완료. 다음 페이지로 진행합니다.
페이지 12 크롤링 완료. 다음 페이지로 진행합니다.
페이지 13 크롤링 완료. 다음 페이지로 진행합니다.
페이지 14 크롤링 완료. 다음 페이지로 진행합니다.
페이지 15 크롤링 완료. 다음 페이지로 진행합니다.
페이지 16 크롤링 완료. 다음 페이지로 진행합니다.
페이지 17 크롤링 완료. 다음 페이지로 진행합니다.
페이지 18 크롤링 완료. 다음 페이지로 진행합니다.
페이지 19 크롤링 완료. 다음 페이지로 진행합니다.
페이지 20 크롤링 완료. 다음 페이지로 진행합니다.
페이지 21 크롤링 완료. 다음 페이지로 진행합니다.
페이지 22 크롤링 완료. 다음 페이지로 진행합니다.
페이지 23 크롤링 완료. 다음 페이지로 진행합니다.
페이지 24 크롤링 완료. 다음 페이지로 진행합니다.
페이지 25 크롤링 완료. 다음 페이지로 진행합니다.
페이지 26 크롤링 완료. 다음 페이지로 진행합니다.
페이지 27 크롤링 완료. 다음 페이지로 진행합니다.
페이지 28 크롤링 완료. 다음 페이지로 진행합니다.
페이지 29 크롤링 완료. 다음 페이지로 진행합니다.
페이지 30 크롤링 완료. 다음 페이지로 진행합니다.
페이지 31 크롤링 완료. 다음 페이지로 진행합니다.
페이지 32 크롤링 완료. 다음 페이지로 진행합니다.

### 레코드 출력 함수
jsonl에 저장된 그대로 출력

print_jsonl_details (파일 경로, num_records)

--

포맷팅 적용한 후 출력


format_jsonl_details(파일 경로, num_reconrds)

In [8]:
import json
import os

def print_jsonl_details(file_path, num_records=5):
    """JSON Lines 파일에서 처음 N개의 레코드를 읽어 세부 내용을 출력합니다."""

    if not os.path.exists(file_path):
        print(f"오류: 파일 '{file_path}'를 찾을 수 없습니다. 경로를 확인해 주세요.")
        return

    print(f"\n--- 📄 '{file_path}' 파일의 처음 {num_records}개 레코드 출력 ---")

    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= num_records:
                break

            try:
               # 1. 각 줄을 JSON 객체로 파싱
              doc = json.loads(line)

              # 2. 필수 필드를 포함하여 출력 형식 지정
              print(f"\n[레코드 #{i+1}]")

              # 💡 올바른 출력 방법: 딕셔너리 전체를 들여쓰기하여 출력
              print(json.dumps(doc, indent=4, ensure_ascii=False))

            except json.JSONDecodeError:
                  print(f"경고: {i+1}번째 줄은 유효한 JSON 형식이 아닙니다. 건너뜁니다.")

# --- 실행 ---
FILE_NAME = 'ssu_rag_data_2025_v2.jsonl'
print_jsonl_details(FILE_NAME, num_records=5)


--- 📄 'ssu_rag_data_2025_v2.jsonl' 파일의 처음 5개 레코드 출력 ---

[레코드 #1]
{
    "source_url": "https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec%82%ac%ed%95%ad/?f&category&paged=1&slug=%EC%88%AD%EC%8B%A4%EB%8C%80%ED%95%99%EA%B5%90-%EA%B8%80%EB%A1%9C%EB%B2%8C%EB%AF%B8%EB%9E%98%EA%B5%90%EC%9C%A1%EC%9B%90-%EB%AF%B8%EB%9E%98%EA%B5%90%EC%9C%A1%ED%8C%80-%EA%B3%84%EC%95%BD%EC%A7%81&keyword",
    "post_title": "숭실대학교 글로벌미래교육원 미래교육팀 계약직원 모집",
    "category": "채용",
    "department": "총무·인사팀",
    "views": 10,
    "posted_date": "2025-11-05",
    "full_body_markdown": "숭실대학교 글로벌미래교육원 미래교육팀에서 아래와 같이 계약직 직원을 모집합니다.  \n모집관련 자세한 사항은 붙임의 모집공고를 참고하여 주시기 바랍니다.\n\n* * *\n\n**1.모집 관련 사항**\n\n**가****.****모집 인원** : 0명 (2025.12.1. 임용 예정)\n\n**나****.****지원 기간** : 2025.11.5.(수) ~ 2025.11.16.(금) 24:00까지 (접수 선착순으로 면접전형 진행)\n\n**다****.****지원 방법** : **이메일 지원** (hsh7403@ssu.ac.kr로 제출서류 발송, _이메일 제목_ _:__숭실대학교 직원 채용 지원_ ____지원자 성명_ _)_\n\n(지원서 내에 모집구분 작성)\n\n**라****.****전형 일정** : 서류전형 → 면접전형(1회) → 최종합격\n\n**마****.****모집 부서 관련사

In [ ]:
import json
import os

def format_print_jsonl_details(file_path, num_records=5):
    """JSON Lines 파일에서 처음 N개의 레코드를 읽어 세부 내용을 출력합니다."""

    if not os.path.exists(file_path):
        print(f"오류: 파일 '{file_path}'를 찾을 수 없습니다. 경로를 확인해 주세요.")
        return

    print(f"\n--- 📄 '{file_path}' 파일의 처음 {num_records}개 레코드 출력 ---")

    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= num_records:
                break

            try:
                # 1. 각 줄을 JSON 객체로 파싱
                doc = json.loads(line)

                # 2. 필수 필드를 포함하여 출력 형식 지정
                print(f"\n[레코드 #{i+1}]")
                print(f"  제목: {doc.get('post_title', 'N/A')}")
                print(f"  URL: {doc.get('source_url', 'N/A')}")
                print(f"  작성일: {doc.get('posted_date', 'N/A')}")

                # 💡 추가/수정된 필드 출력
                print(f"  부서: {doc.get('department', 'N/A')}")
                print(f"  조회수: {doc.get('views', 'N/A')}회") # views는 이제 숫자 (int)
                # doc.get('status')는 데이터 구조에서 제외했으므로 출력하지 않습니다.

                # 3. 첨부 파일 및 이미지 정보 출력
                image_urls = doc.get('image_urls', [])
                download_links = doc.get('download_links', [])

                print(f"  이미지 수: {len(image_urls)}개")
                print(f"  첨부파일 수: {len(download_links)}개")

                # 첨부 파일 목록이 있다면 자세히 출력
                if download_links:
                    print("    [첨부파일 목록]")
                    for link_item in download_links[:2]: # 최대 2개만 보여줍니다.
                        print(f"      - {link_item.get('text', '파일명 없음')}")

                # 4. 본문 텍스트는 너무 길 수 있으므로 일부만 출력
                body_text = doc.get('full_body_markdown', '본문 없음')

                # 본문이 길 경우 처음 200자만 출력
                display_text = body_text[:200].replace('\n', ' ') + ('...' if len(body_text) > 200 else '')

                print(f"  본문 미리보기: {display_text}")

            except json.JSONDecodeError:
                print(f"경고: {i+1}번째 줄은 유효한 JSON 형식이 아닙니다. 건너뜁니다.")

# --- 실행 ---
FILE_NAME = 'ssu_rag_data_2025_v2.jsonl'
format_print_jsonl_details(FILE_NAME, num_records=5)


--- 📄 'ssu_rag_data_2025_v2.jsonl' 파일의 처음 5개 레코드 출력 ---

[레코드 #1]
  제목: 2026년도 (재)전기공사공제조합장학회 제21기 장학생 선발 안내
  URL: https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec%82%ac%ed%95%ad/?f&category&paged=1&slug=2026%EB%85%84%EB%8F%84-%EC%9E%AC%EC%A0%84%EA%B8%B0%EA%B3%B5%EC%82%AC%EA%B3%B5%EC%A0%9C%EC%A1%B0%ED%95%A9%EC%9E%A5%ED%95%99%ED%9A%8C-%EC%A0%9C21%EA%B8%B0-%EC%9E%A5%ED%95%99%EC%83%9D-%EC%84%A0&keyword
  작성일: 2025-11-04
  부서: 장학팀
  조회수: 34회
  이미지 수: 1개
  첨부파일 수: 0개
  본문 미리보기: [![](https://scatch.ssu.ac.kr/wp-content/uploads/sites/5/2025/11/2026년도-재전기공사공제조합장학회-제21기-장학생-선발-공고.jpg)](https://scatch.ssu.ac.kr/wp-content/uploads/sites/5/2025/11/2026년도-재전기공사공제조합장학회-제21기-장학생-선발-공고...

[레코드 #2]
  제목: [중앙도서관] ‘천선란’ 작가 저자강연회 참가자 모집(~11/21)
  URL: https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec%82%ac%ed%95%ad/?f&category&paged=1&slug=%EC%A4%91%EC%95%99%EB%8F%84%EC%84%9C%EA%B4%80-%EC%B2%9C%EC%84%A0%EB%9E%80-%EC%9E%91%EA%B0%80-%EC%A0%80%EC%9E%90%EA%B0%95%EC%97%B0%ED%9A%8C-%EC%B0%B8%EA%B0%80%EC%9E%90-%EB

## 마크다운 텍스트에서 남아있는 링크 (img, downloads, 또는 모든 https://)를 분리
- cleaned_markdown: 링크 제거한 마크다운
- image_urls: 이미지 url 저장
- downloads_links: 다운로드 링크 저장
- extra_links: 그외 본문에 삽입된 링크

In [11]:
import re

def clean_markdown_and_extract_links(markdown_text):
    """
    Markdown 텍스트에서 링크를 추출/분류하고, 링크 텍스트가 비정상적인 경우
    모두 빈 문자열("")로 처리하여 본문에서 제거합니다.
    """
    if not markdown_text:
        return {
            'cleaned_markdown': '',
            'image_urls': [],
            'download_links': [],
            'extra_links': []
        }

    image_urls = []
    download_links = []
    extra_links = []

    # 1. 모든 Markdown 링크 패턴을 찾습니다: [텍스트](URL) 또는 ![텍스트](URL)
    link_pattern = re.compile(r'\!?\[(.*?)\]\((.*?)\)', re.DOTALL)

    # --- A. Markdown 링크 추출 및 분류 ---
    for match in link_pattern.finditer(markdown_text):
        raw_link_text = match.group(1).strip()
        url = match.group(2).strip()
        url_lower = url.lower()

        # 1. 💡 링크 텍스트 클리닝: 마크다운 서식 제거 (**, *, #, 줄바꿈 등)
        cleaned_link_text = re.sub(r'(\*\*|#|\*|_|\n|>>)', '', raw_link_text).strip()

        # 2. 💡 최종 Link Text 결정: 텍스트가 너무 길거나 없으면 빈 문자열("") 사용
        # (200자를 초과하면 본문 텍스트가 링크 텍스트로 오염된 것으로 간주)
        if not cleaned_link_text or len(cleaned_link_text) > 200:
            final_link_text = "" # 👈 빈 문자열로 확정
        else:
            final_link_text = cleaned_link_text

        # --- 분류 로직 (A, B, C) ---
        if url_lower.endswith(('.jpg', '.jpeg', '.png', '.gif', '.svg', '.webp')):
            # A. 이미지 URL
            if url not in image_urls:
                image_urls.append(url)

        elif url_lower.endswith(('.pdf', '.zip', '.hwp', '.doc', '.docx')):
            # B. 다운로드 링크
            if url not in [dl['href'] for dl in download_links]:
                download_links.append({'text': final_link_text, 'href': url})

        elif url_lower.startswith(('http://', 'https://')):
            # C. 일반 외부 링크 (extra_links)
            if url not in [el['href'] for el in extra_links]:
                extra_links.append({'text': final_link_text, 'href': url})

    # --- B. 본문에서 링크 정보 제거 및 순수 URL 텍스트 추출/제거 ---

    # 3-1. Markdown 링크 패턴 제거
    cleaned_markdown = re.sub(r'\!?\[.*?\]\([^\)]+\)', '', markdown_text)

    # 3-2. 순수 URL 텍스트 추출 및 extra_links에 추가 (Sic History, 네이버 카페 등)
    url_pattern_raw = re.compile(r'(https?:\/\/\S+)', re.IGNORECASE)

    temp_markdown = cleaned_markdown # URL 추출은 임시 변수를 사용

    for match in url_pattern_raw.finditer(temp_markdown):
        url = match.group(1).strip()

        # 💡 이미 분류된 링크가 아니라면 extra_links에 추가 (text: "")
        if url not in [u for url_list in [image_urls] + [d['href'] for d in download_links] + [e['href'] for e in extra_links] for u in url_list]:
             # 텍스트가 없으므로 빈 문자열을 사용
            extra_links.append({'text': "", 'href': url})

        # 3-3. 본문에서 모든 URL 텍스트 제거
        cleaned_markdown = cleaned_markdown.replace(url, ' ')

    # --- 4. 잔여물 및 공백 정리 ---
    cleaned_markdown = re.sub(r'[\(\)\[\]_—*]', '', cleaned_markdown)
    cleaned_markdown = re.sub(r'[ \t]+', ' ', cleaned_markdown)
    cleaned_markdown = re.sub(r'\n{2,}', '\n\n', cleaned_markdown).strip()

    return {
        'cleaned_markdown': cleaned_markdown,
        'image_urls': image_urls,
        'download_links': download_links,
        'extra_links': extra_links
    }

In [13]:
import json
import os
import re
# MLLM 라이브러리 및 순수 텍스트 계산 함수는 외부에서 정의되었다고 가정합니다.


# =================================================================
# 📌 1단계: 링크 정리 및 메타데이터 확정 함수
# =================================================================
def stage1_process_links(input_file_path, output_file_path):
    """
    1단계: JSONL 파일을 읽어 링크를 추출/분류하고,
    full_body_markdown에서 링크를 제거하며,
    extra_links 필드를 포함하여 새 파일에 저장합니다.
    """
    if not os.path.exists(input_file_path):
        print(f"❌ 오류: 입력 파일 '{input_file_path}'를 찾을 수 없습니다.")
        return

    total_processed = 0
    print(f"--- 1단계 시작: 링크 정리 및 메타데이터 확정 ---")

    with open(input_file_path, 'r', encoding='utf-8') as infile, \
         open(output_file_path, 'w', encoding='utf-8') as outfile:

        for i, line in enumerate(infile):
            try:
                raw_doc = json.loads(line)
                markdown_text = raw_doc.get('full_body_markdown', '')

                # 💡 1. 기존 링크 정보 백업
                existing_image_urls = raw_doc.get('image_urls', [])
                existing_download_links = raw_doc.get('download_links', [])
                existing_extra_links = raw_doc.get('extra_links', []) # 👈 extra_links 필드 추가

                # 2. 링크 추출 및 Markdown 정리 함수 호출
                # clean_markdown_and_extract_links는 'cleaned_markdown', 'image_urls', 'download_links', 'extra_links' 반환 가정
                link_results = clean_markdown_and_extract_links(markdown_text)

                # 3. Raw Document 필드 업데이트 및 병합

                # A. 콘텐츠 필드 (덮어쓰기)
                raw_doc['cleaned_markdown'] = link_results['cleaned_markdown']

                # B. image_urls 병합 (Set 사용)
                all_images = set(existing_image_urls)
                all_images.update(link_results['image_urls'])
                raw_doc['image_urls'] = list(all_images)

                # C. download_links 병합 (hrefs 기준 중복 제거)
                existing_download_hrefs = {item['href'] for item in existing_download_links}
                new_download_links = existing_download_links[:]

                for new_link in link_results['download_links']:
                    if new_link['href'] not in existing_download_hrefs:
                        new_download_links.append(new_link)
                raw_doc['download_links'] = new_download_links

                # D. 💡 extra_links 병합 (hrefs 기준 중복 제거)
                existing_extra_hrefs = {item['href'] for item in existing_extra_links}
                new_extra_links = existing_extra_links[:]

                for new_link in link_results['extra_links']:
                    if new_link['href'] not in existing_extra_hrefs:
                        new_extra_links.append(new_link)

                raw_doc['extra_links'] = new_extra_links # 👈 최종 필드 업데이트

                # 4. 최종 데이터 저장
                outfile.write(json.dumps(raw_doc, ensure_ascii=False) + '\n')
                total_processed += 1

            except json.JSONDecodeError:
                print(f"경고: {i+1}번째 레코드 파싱 오류. 건너뜁니다.")
            except Exception as e:
                print(f"경고: {i+1}번째 레코드 처리 중 오류 발생 ({e}). 레코드 건너뜁니다.")

    print(f"--- 🎉 1단계 완료: {total_processed}개 레코드 처리 완료 ---")
    print(f"결과 파일: '{output_file_path}'에 저장되었습니다.")

In [19]:
# =================================================================
# 📌 실행 파이프라인
# =================================================================

INPUT_FILE_RAW = "/content/drive/MyDrive/자연어/ssu_rag_data_2025_v2.jsonl" # 크롤링 직후 파일
INTERMEDIATE_FILE = '/content/drive/MyDrive/자연어/stage1_cleaned_data.jsonl' # 1단계 중간 파일
OUTPUT_FILE_FINAL = '/content/drive/MyDrive/자연어/ssu_rag_data_2025_v3.jsonl'    # 최종 RAG 입력 파일

# 1. 🔗 링크 정리 및 메타데이터 확정
stage1_process_links(INPUT_FILE_RAW, INTERMEDIATE_FILE)

# 중간 결과 확인
FILE_NAME = '/content/drive/MyDrive/자연어/stage1_cleaned_data.jsonl'
print_jsonl_details(FILE_NAME, num_records=5)

--- 1단계 시작: 링크 정리 및 메타데이터 확정 ---
--- 🎉 1단계 완료: 1112개 레코드 처리 완료 ---
결과 파일: '/content/drive/MyDrive/자연어/stage1_cleaned_data.jsonl'에 저장되었습니다.

--- 📄 '/content/drive/MyDrive/자연어/stage1_cleaned_data.jsonl' 파일의 처음 5개 레코드 출력 ---

[레코드 #1]
{
    "source_url": "https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec%82%ac%ed%95%ad/?f&category&paged=1&slug=%EC%88%AD%EC%8B%A4%EB%8C%80%ED%95%99%EA%B5%90-%EA%B8%80%EB%A1%9C%EB%B2%8C%EB%AF%B8%EB%9E%98%EA%B5%90%EC%9C%A1%EC%9B%90-%EB%AF%B8%EB%9E%98%EA%B5%90%EC%9C%A1%ED%8C%80-%EA%B3%84%EC%95%BD%EC%A7%81&keyword",
    "post_title": "숭실대학교 글로벌미래교육원 미래교육팀 계약직원 모집",
    "category": "채용",
    "department": "총무·인사팀",
    "views": 12,
    "posted_date": "2025-11-05",
    "full_body_markdown": "숭실대학교 글로벌미래교육원 미래교육팀에서 아래와 같이 계약직 직원을 모집합니다.  \n모집관련 자세한 사항은 붙임의 모집공고를 참고하여 주시기 바랍니다.\n\n* * *\n\n**1.모집 관련 사항**\n\n**가****.****모집 인원** : 0명 (2025.12.1. 임용 예정)\n\n**나****.****지원 기간** : 2025.11.5.(수) ~ 2025.11.16.(금) 24:00까지 (접수 선착순으로 면접전형 진행)\n\n**다****.****지원 방법** : **이메

## cleaned_markdown 에서 순수 텍스트 길이가 부족한 레코드에서 이미지 캡셔닝을 시행
- 공지사항 중 사진만 올라와 있는 경우가 있음.
- 이 경우 gemini flash 모델을 사용하여 이미지에서 정보를 추출함.

In [20]:
# --- B. 순수 텍스트 길이 계산 함수 ---
def calculate_pure_text_length(markdown_text):
    """Markdown 마크업을 제거한 순수 텍스트의 길이를 계산합니다."""
    if not markdown_text:
        return 0
    # Markdown 서식 (**, *, #) 제거
    text = re.sub(r'(\*\*|#|\*)', '', markdown_text)
    text = re.sub(r'&[a-z0-9]+;', ' ', text) # HTML 엔티티 제거
    return len(text.strip())

In [25]:
import os
from google import genai
import requests

# 🚨 보안 경고: 이 코드를 GitHub 등에 푸시하면 안 됩니다.
# 현재 세션에서만 임시로 환경 변수를 설정합니다.
os.environ['GEMINI_API_KEY'] = 'YOUR_PRIVATE_KEY'
print("✅ GEMINI_API_KEY가 현재 환경 변수에 설정되었습니다.")

# 클라이언트가 제대로 초기화되는지 확인
try:
    client = genai.Client()
    print("✅ Gemini 클라이언트 초기화 성공.")
except Exception as e:
    print(f"❌ Gemini 클라이언트 초기화 실패: {e}")

import base64
from urllib.parse import urlparse, quote
from google.genai import types

# ----------------------------------------------------
# 💡 URL 인코딩 함수 (한글 URL 처리)
# ----------------------------------------------------
def encode_korean_url(url):
    """URL의 경로와 쿼리 부분에 있는 한글을 퍼센트 인코딩합니다."""
    parsed_url = urlparse(url)
    # 경로와 쿼리 부분만 인코딩 (호스트명이나 스키마는 인코딩하지 않음)
    path = quote(parsed_url.path, safe='/')
    query = quote(parsed_url.query, safe='=&')

    encoded_url = parsed_url._replace(path=path, query=query).geturl()
    return encoded_url

# ----------------------------------------------------
# 🤖 Gemini Base64 캡셔닝 함수 (최종 MLLM 호출)
# ----------------------------------------------------

import time # 💡 time 모듈 추가
import os
import requests
import base64
from google import genai
from google.genai import types
# (주의: calculate_pure_text_length 함수, MIN_TEXT_LENGTH 정의 필요)

MAX_RETRIES = 5

def generate_image_caption_base64(image_url):
    """
    이미지 캡셔닝을 수행하고, API 오류 발생 시 재시도 로직을 포함합니다.
    """

    for attempt in range(MAX_RETRIES):
        try:
            client = genai.Client()

            # 1. 이미지 다운로드 및 Base64 인코딩
            img_response = requests.get(image_url, stream=True)
            img_response.raise_for_status()
            image_bytes = img_response.content

            mime_type = img_response.headers.get('Content-Type')
            if not mime_type or not mime_type.startswith('image/'):
                 mime_type = 'image/jpeg'

            base64_image = base64.b64encode(image_bytes).decode("utf-8")

            prompt_text = (
                "이 문서를 상세히 분석하여 표, 일정, 날짜 등 모든 정보를 OCR을 통해 추출하고 "
                "키:값 형태로 구조화해 주세요. 이 이미지는 학교 공지사항 포스터입니다."
            )

            # 2. 모델 입력 정의 (안정적인 딕셔너리 구조 사용)
            model_input = [
                {"inline_data": {"mime_type": mime_type, "data": base64_image}},
                {"text": prompt_text}
            ]

            # 3. API 호출
            response = client.models.generate_content(
                model='gemini-2.5-flash',
                contents=model_input,
            )

            # 호출 성공 시 즉시 반환
            return response.text.strip()

        except Exception as e:
            error_message = str(e)

            # 4. 서버 오류일 경우 재시도 판단
            if "503 UNAVAILABLE" in error_message or "Resource Exhausted" in error_message or "429" in error_message:
                if attempt < MAX_RETRIES - 1:
                    wait_time = 2 ** attempt  # 1초, 2초, 4초...
                    print(f"⚠️ API 서버 오류 ({error_message[:30]}...). {wait_time}초 후 재시도 ({attempt + 1}/{MAX_RETRIES})")
                    time.sleep(wait_time) # 💡 time.sleep()이 이제 작동합니다.
                    continue
                else:
                    # 최대 재시도 횟수를 초과한 경우
                    raise RuntimeError(f"Gemini API 최종 실패 (재시도 횟수 초과): {error_message}")

            # 다운로드 실패 등 다른 치명적인 오류일 경우
            raise RuntimeError(f"처리 실패: {error_message}")

    raise RuntimeError("알 수 없는 이유로 Gemini API 호출 실패.")


✅ GEMINI_API_KEY가 현재 환경 변수에 설정되었습니다.
✅ Gemini 클라이언트 초기화 성공.


In [26]:

# =================================================================
# 📌 2단계: 조건부 이미지 캡셔닝 및 텍스트 통합 함수
# =================================================================

def stage2_process_images_resumable(input_file_path, output_file_path, min_text_length=50):

    # 1. ✅ 이미 처리된 URL 목록 로드 (재개 지점 파악)
    processed_urls = set()
    if os.path.exists(output_file_path):
        with open(output_file_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    doc = json.loads(line)
                    # source_url을 사용하여 처리 여부를 판단합니다.
                    if doc.get('source_url'):
                        processed_urls.add(doc['source_url'])
                except json.JSONDecodeError:
                    pass

    total_processed_initial = len(processed_urls)
    total_processed_new = 0
    total_captioned = 0

    print(f"--- 🔄 RAG 처리 재개 시작: 이미 {total_processed_initial}개 레코드 처리 완료 ---")

    error_occurred = False
    try:
        # 2. 📝 파일 열기: 출력 파일은 'a' (append, 추가) 모드로 열어야 기존 내용을 덮어쓰지 않습니다.
        with open(input_file_path, 'r', encoding='utf-8') as infile, \
             open(output_file_path, 'a', encoding='utf-8') as outfile:

            for i, line in enumerate(infile):
                try:
                    raw_doc = json.loads(line)
                    source_url = raw_doc.get('source_url')

                    # 3. 🛑 처리 건너뛰기 조건: 이미 처리된 URL이면 continue
                    if source_url in processed_urls:
                        continue # 다음 레코드로 이동 (중단 지점까지 빠르게 건너뜀)

                    markdown_text = raw_doc.get('cleaned_markdown', '')
                    image_urls = raw_doc.get('image_urls', [])
                    raw_doc['captioned'] = False

                    pure_text_length = calculate_pure_text_length(markdown_text)

                    # 4. 🖼️ 캡셔닝 조건 확인 및 실행
                    if image_urls and pure_text_length < min_text_length:
                        caption_list = []
                        print(f"⚠️ 레코드 #{i+1}: 텍스트 부족 ({pure_text_length}자). 캡셔닝 실행.")

                        for img_url in image_urls:
                            # generate_image_caption_base64 함수는 실패 시 RuntimeError를 발생시킵니다.
                            caption = generate_image_caption_base64(img_url)
                            caption_list.append(caption)

                        # 5. 캡션 통합
                        caption_text = "\n\n### 이미지 분석 결과 ###\n" + "\n".join(caption_list)
                        raw_doc['cleaned_markdown'] += caption_text
                        raw_doc['captioned'] = True
                        total_captioned += 1

                    # 6. 최종 데이터 저장 (append 모드로 파일 끝에 추가됨)
                    outfile.write(json.dumps(raw_doc, ensure_ascii=False) + '\n')
                    total_processed_new += 1

                except RuntimeError:
                    # API 호출 실패로 발생한 RuntimeError를 잡고 루프를 종료합니다.
                    error_occurred = True
                    raise
                except json.JSONDecodeError:
                    continue


    except RuntimeError:
        # 중단 지점까지의 모든 데이터는 파일에 안전하게 저장됩니다.
        print(f"\n✅ 파이프라인 중단. 중단 지점까지의 모든 데이터가 '{output_file_path}'에 안전하게 저장되었습니다.")
        raise # 이 예외를 외부로 전파하여 프로그램이 종료되도록 합니다.

    print(f"--- 🎉 2단계 완료 ---")
    print(f"총 레코드 수: {total_processed_initial + total_processed_new}개, 캡셔닝 적용: {total_captioned}개")
    print(f"결과 파일: '{output_file_path}'에 저장되었습니다.")
    return




In [27]:
import time
import os
# (stage2_process_images_resumable 함수 및 기타 필요 함수는 정의되어 있다고 가정)

# 📌 파일 경로 설정
INTERMEDIATE_FILE = '/content/drive/MyDrive/자연어/stage1_cleaned_data.jsonl' # 1단계 중간 파일
OUTPUT_FILE_FINAL = '/content/drive/MyDrive/자연어/ssu_rag_data_2025_v3.jsonl'    # 최종 RAG 입력 파일
MAX_ATTEMPTS = 3 # 최대 재시도 횟수

def run_rag_pipeline_with_auto_retry():
    """
    RAG 캡셔닝 파이프라인을 실행하고, 치명적인 오류(API 과부하 등) 발생 시 재시도합니다.
    """

    print("--- 🚀 RAG 캡셔닝 파이프라인 자동 재실행 시작 ---")

    # ⚠️ 이전에 실행된 1단계 함수 (링크 정리)는 성공했다고 가정합니다.
    # stage1_process_links(INPUT_FILE_RAW, INTERMEDIATE_FILE)

    for attempt in range(1, MAX_ATTEMPTS + 1):
        print(f"\n[시도 #{attempt} / {MAX_ATTEMPTS}]")

        try:
            # 1. 재개 함수 호출 (중단된 지점부터 자동으로 처리)
            stage2_process_images_resumable(INTERMEDIATE_FILE, OUTPUT_FILE_FINAL)

            # 2. 함수가 오류 없이 완료되면 루프 종료
            print("🎉 파이프라인이 성공적으로 완료되었습니다!")
            return

        except RuntimeError as e:
            # 3. API 오류 (RuntimeError) 발생 시
            if attempt < MAX_ATTEMPTS:
                # 💡 stage2 함수에서 이미 저장 메시지를 출력했으므로 여기서는 대기 메시지만 출력합니다.
                wait_time = 15 * attempt
                print(f"**잠시 후 재시도합니다. {wait_time}초 대기 중...**")
                time.sleep(wait_time)
            else:
                # 4. 최대 재시도 횟수 초과 시 최종 중단
                print(f"\n\n❌ 최대 재시도 횟수({MAX_ATTEMPTS}회) 초과. 파이프라인을 최종 중단합니다.")
                print(f"최종 오류: {e}")
                break # 루프 종료

if __name__ == '__main__':
    run_rag_pipeline_with_auto_retry()

--- 🚀 RAG 캡셔닝 파이프라인 자동 재실행 시작 ---

[시도 #1 / 3]
--- 🔄 RAG 처리 재개 시작: 이미 864개 레코드 처리 완료 ---
⚠️ 레코드 #865: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #866: 텍스트 부족 (1자). 캡셔닝 실행.
⚠️ 레코드 #867: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #870: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #874: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #876: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #879: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #881: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #886: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #887: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #890: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #896: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #910: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #913: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #916: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #917: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #930: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #932: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #934: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #940: 텍스트 부족 (46자). 캡셔닝 실행.
⚠️ 레코드 #941: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #943: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #945: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #953: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #958: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #959: 텍스트 부족 (25자). 캡셔닝 실행.
⚠️ 레코드 #960: 텍스트 부족 (0자).

### 이미지 캡셔닝 수행 확인
- 10개 레코드만 임시파일에 저장해두고 캡셔닝 후 출력

In [ ]:
import json
import os

# ⚠️ 이 경로는 사용자의 실제 파일 경로와 일치해야 합니다.
ORIGINAL = INTERMEDIATE_FILE
OUTPUT_FILE_TEST = 'test_input_10.jsonl'
NUM_RECORDS = 10

def create_test_file(input_path, output_path, num_records):
    """
    원본 JSONL 파일에서 처음 N개의 레코드만 복사하여 테스트 파일을 생성합니다.
    """
    if not os.path.exists(input_path):
        print(f"❌ 오류: 원본 파일 '{input_path}'를 찾을 수 없습니다. 1단계 처리가 완료되었는지 확인하세요.")
        return

    total_copied = 0

    print(f"\n--- 📋 테스트 파일 생성 시작 (첫 {num_records}개 레코드) ---")

    with open(input_path, 'r', encoding='utf-8') as infile, \
         open(output_path, 'w', encoding='utf-8') as outfile:

        for i, line in enumerate(infile):
            if i >= num_records:
                break

            # 레코드가 유효한 JSON 형식인지 확인 후 복사
            try:
                # JSON 파싱 및 다시 문자열화 (안전한 복사를 위해)
                doc = json.loads(line)
                outfile.write(json.dumps(doc, ensure_ascii=False) + '\n')
                total_copied += 1
            except json.JSONDecodeError:
                print(f"경고: {i+1}번째 레코드 파싱 오류. 복사되지 않았습니다.")
                continue

    print(f"--- ✅ 테스트 파일 생성 완료 ---")
    print(f"'{output_path}' 파일에 총 {total_copied}개 레코드가 저장되었습니다.")

# --- 실행 ---
create_test_file(ORIGINAL, OUTPUT_FILE_TEST, NUM_RECORDS)


--- 📋 테스트 파일 생성 시작 (첫 10개 레코드) ---
--- ✅ 테스트 파일 생성 완료 ---
'test_input_10.jsonl' 파일에 총 10개 레코드가 저장되었습니다.


In [ ]:
stage2_process_images(OUTPUT_FILE_TEST, 'test_result_10.jsonl')
check_specific_caption_results_pandas('test_result_10.jsonl')

--- 🖼️ 2단계 시작: 조건부 이미지 캡셔닝 ---
⚠️ 레코드 #1: 텍스트 부족 (0자). 캡셔닝 실행.
⚠️ 레코드 #4: 텍스트 부족 (0자). 캡셔닝 실행.
--- 🎉 2단계 완료 ---
총 레코드 수: 10개, 캡셔닝 적용: 2개
결과 파일: 'test_result_10.jsonl'에 저장되었습니다.
✅ 데이터 로드 성공. 총 10개 레코드 확인.

--- 🖼️ 요청된 10개 레코드 확인 ---

[레코드 #1 / 인덱스 0]
post_title: 2026년도 (재)전기공사공제조합장학회 제21기 장학생 선발 안내
cleaned_markdown: 

### 이미지 분석 결과 ###
다음은 이미지에서 추출된 정보이며, OCR을 통해 분석된 표, 일정, 날짜 등의 모든 데이터를 키:값 형태로 구조화한 것입니다.

**공지사항 제목:** (재)전기공사공제조합장학회 2026년도 제21기 장학생 선발 공고
**공지 내용:** 전기산업의 미래를 이끌어갈 인재를 지원하기 위해 전기공사공제조합장학회가 장학생을 선발합니다.
**장학금 지원 성격:** 등록금 실비 지원 (등록금성장학금)

---

### 선발개요

*   **접수기간:** 2025. 11. 3.(월) 9시 ~ 2025. 11. 14.(금) 18시
*   **접수방법:**
    *   온라인 원서접수(www.ecfcsf.or.kr)만 가능
    *   이메일·우편방문접수는 불가합니다.
*   **선발인원:** 대학생 74명 (조합원전형, 일반전형)
*   **선발기준:** 서류심사 (학업성적 및 가정형편)
*   **합격발표:** 2025년 12월 중 (일정 변경 시 장학회 홈페이지 공지)

---

### 장학혜택

*   **지원금액:**
    *   등록금 연간 최대 5백만 원 지원 (학기당 250만 원 한도)
    *   등록금에서 교내 및 국가장학금을 제한 금액을 학기당 250만원 한도 내 지원
*   **지급기간:** 2026년도 1학기 및 2학기(정규학기에 한정)
    *   계